# L1: Information retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

**⚠️ Make sure that you have read the [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating) before starting with this lab.** 

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [4]:
import bz2
import pandas as pd

with bz2.open('app-descriptions.json.bz2') as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. You can access data in a DataFrame in various ways, including by row and column. To give an example, the code in the next cell shows rows 200–204:

In [5]:
df[200:205]

,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to access fields from the description column.

In [6]:
df['description'][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any transformation applied to a text before vectorisation. Here you can restrict yourself to a simple type of preprocessing: tokenisation, stop word removal, and lemmatisation.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure to read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you deem necessary.

In [39]:
import spacy
def preprocess(text):
    # TODO: Replace the next line with your own code.
    
    nlp = spacy.load('en_core_web_sm')
    text_pre=[]
    for x in nlp(text):
        if x.is_stop==False and x.is_alpha==True:
            text_pre.append(x.lemma_)
    
    return text_pre

Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses the given text by tokenising it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

To speed up the preprocessing, you should disable loading those spaCy components you do not need, such as the parser, and named entity recogniser. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

### 🤞 Test your code

Test your implementation by running the following cell:

In [40]:
preprocess('Apple is looking at buying U.K. startup for $1 billion')

['Apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


## Problem 2: Vectorising

Your next task is to vectorise the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectoriser. (In scikit-learn terminology, the `preprocessor` handles string-level preprocessing.)

In [98]:
import numpy as np

# TODO: Replace the next line with your own code.
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=preprocess)
X = vectorizer.fit_transform(df['description'])


### 🤞 Test your code

Test your implementation by running the following cell:

In [86]:
X.shape

(1614, 21396)

This should show the dimensions of the matrix `X` to be around 1614 × 21389. (The number of rows should be exactly 1614. The number of columns may differ from that given here depending on the version of spaCy and the version of the language model used, as well as the pre-processing.)

## Problem 3: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning and allows you to easily find a predefined number of app descriptions whose vector representations are closest to the query vector.

In [107]:
from sklearn.neighbors import NearestNeighbors
def search(query):
    # TODO: Replace the next line with your own code.
    query_vector=vectorizer.transform([query])
    neigh=NearestNeighbors(n_neighbors=10,metric='cosine')
    neigh.fit(X)
    index=neigh.kneighbors(query_vector,return_distance=False)[0]
    return df.iloc[index,:]

Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

### 🤞 Test your code

Test your implementation by running the following cell:

In [108]:
search('dodge trains')

,name,description
1301,Subway Surfers,DASH as fast as you can! \nDODGE the oncoming ...
1428,Train Conductor World,Master and manage the chaos of international r...
1394,Tiny Rails,All aboard for an adventure around the world!\...
1300,Subway Princess Runner,"Subway princess runner, Bus run, forest rush w..."
998,No Humanity - The Hardest Game,2M+ Downloads All Over The World!\n\n* IGN Nom...
1429,Train for Animals - BabyMagica free,"🚂 BabyMagica ""Train for Animals"" is a educatio..."
1168,Rush,Are you ready for a thrilling ride?\n\nRush th...
786,LEGO® DUPLO® Train,All aboard! Driving the colorful LEGO® DUPLO® ...
1465,Virus War - Space Shooting Game,Warning! Virus invasion! Destroy them with you...
1153,Road Riot,Road Riot is the global sensation that defined...


The top hit in the list should be *Subway Surfers*.

## Problem 4: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower, the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms from the app descriptions have the lowest/highest idf.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [198]:
# TODO: Replace the next line with your own code.
sorted_terms=sorted(zip(vectorizer.get_feature_names_out(),vectorizer.idf_),key=lambda x: (x[1], x[0]))
terms = [x[0] for x in sorted_terms]

Now, print the 10 terms with the lowest/highest idf. How do you explain the results?

In [199]:
# print the 10 terms with lowest idf
print(terms[:10])
# print the 10 terms with highest idf
#print(terms[::-1][:10])

['game', 'play', 'feature', 'free', 'new', 'world', 'time', 'app', 'fun', 'use']


## Problem 5: Keyword extraction

We often want to extract salient keywords from a document. A simple method is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a text.

In [219]:
def keywords(text, n=10):
    # TODO: Replace the next line with your own code.
    sorted_words=sorted(zip(vectorizer.get_feature_names_out(),vectorizer.transform([text]).A[0]),key=lambda x: (x[1], x[0]))[::-1][:n]
    keywords=[x[0] for x in sorted_words]
    return keywords

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

### 🤞 Test your code

Test your implementation by running the following cell:

In [220]:
print(keywords(df['description'][1428]))

['train', 'railway', 'railroad', 'rail', 'chaos', 'crash', 'tram', 'timetable', 're', 'railyard']


This should give output similar to the following:

The exact output may differ slightly depending on the strategy used to break ties.

## Reflection questions

The following reflection questions will help you prepare for the diagnostic test. Answer each of them in the form of a short text and put your answers in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 1.1:** Why do we remove common stop words and lemmatise the text? Can you give an example of a scenario where, in addition to common stopwords, there are also *application-specific* stop words?

**RQ 1.2:** In Problem&nbsp;2, what do the dimensions of the matrix `X` correspond to? This matrix gives rise to different types of *representations*. Explain what these representations are. What information from the data is preserved, what information is lost in these representations?

**RQ 1.3:** What does it mean that a term has a high/low idf value? Based on this, how can we use idf to automatically identify stop words? Why do you think is idf not used as a term weighting on its own, but always in connection with term frequency (tf–idf)?

*TODO: Enter your answers here*

**Congratulations on finishing L1! 👍**